## Print out available UWLCM parameters

In [ ]:
%%bash
module load GCC/11.3.0 OpenMPI/4.1.4-mpi-thread-multiple Boost.MPI/1.79.0-mpi-thread-multiple HDF5/1.12.2-threadsafe CMake/3.23.1 Blitz++/1.0.2 2>/dev/null
LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/net/pr2/projects/plgrid/plgguwicmw/geolab/lib/" mpiexec -N 1 /net/pr2/projects/plgrid/plgguwicmw/geolab/bin/uwlcm --micro=lgrngn --help

## Simulation

### Set simulation parameters

In [ ]:
import os.path

In [ ]:
outdir = os.path.expandvars("$SCRATCH/outdir_test/")

#### blk_1m microphysics

In [ ]:
rt_params = {
    "outdir" : outdir,
    "case"   : "moist_thermal",
    "nx"     : "121",
    "ny"     : "0", # ny>0 makes the simulation 3D
    "nz"     : "121",
    "dt"     : "1",
    "nt"     : "600",
    "micro"  : "blk_1m",
    "outfreq": "10"
}

#### Lagrangian microphysics

In [ ]:
# 91 x 91 x 61 nt 600 takes about 20 min on 16 cores and 1 GPU
rt_params = {
    "outdir" : outdir,
    "case"   : "moist_thermal",
    "nx"     : "181",
    "ny"     : "0", # ny>0 makes the simulation 3D
    "nz"     : "121",
    "dt"     : "1",
    "nt"     : "600",
    "micro"  : "lgrngn",
    "outfreq": "30",
    "outwindow" : "1",
    "sgs"       : "1",
    "sd_conc"   : "1000",
    "sstp_cond"  : "2",
    "sstp_coal"  : "2",
    "coal"  : "0", # don't model collision-coalescence
    "case_n_stp_multiplier" : "1",
    "aerosol_independent_of_rhod" : "1",
    "backend" : "CUDA" # use multi_CUDA in case of multiple GPUs
}

In [ ]:
# Converting dictionary to string
rt_params_str = " ".join(f"\-\-{k}={v}" for k, v in rt_params.items())

### Run the model

In [ ]:
%%bash -s "{rt_params_str}"
module load GCC/11.3.0 OpenMPI/4.1.4-mpi-thread-multiple Boost.MPI/1.79.0-mpi-thread-multiple HDF5/1.12.2-threadsafe CMake/3.23.1 Blitz++/1.0.2 2>/dev/null
LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/net/pr2/projects/plgrid/plgguwicmw/geolab/lib/" mpiexec -quiet -N 1 /net/pr2/projects/plgrid/plgguwicmw/geolab/bin/uwlcm ${1//\\}
# LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/net/people/plgrid/plgpdziekan/builds_with_mpi/lib/" mpiexec -N 1 /net/people/plgrid/plgpdziekan/builds_with_mpi/bin/uwlcm ${1//\\}

## Result analysis

In [ ]:
import importlib
import matplotlib.pyplot as plt
import numpy as np
import xrUWLCM as xrU
import math

#from xhistogram.xarray import histogram
import xarray as xr

In [ ]:
# load the data
data, data_DSD = xrU.load_outdir(outdir)
data = xrU.calc_all(data).pipe(xrU.convert_units)

### Print DataSet content

In [ ]:
data #variables of particular interest: nc, nr, rc, rr, cloud_r_mean, rain_r_mean, cloud_sigma, rain_sigma

### Example plots

In [ ]:
# showing cloud water content
time = np.array([[60., 120, 180, 240], [300, 360, 420, 480]]) # [s]
nrow = len(time)
ncol = 0
for row in time:
    ncol = max(ncol, len(row))
fig, axs = plt.subplots(nrow, ncol, figsize = (ncol*3, nrow*3))
#data.rain_r_mean.sel(t=time/3600, method='nearest').plot(x='x')
for i, t in enumerate(time.flatten()):
    ax = axs.flatten()[i]
    rc = data.rc.sel(t=t/3600, method='nearest').plot(x='x', ax=ax, add_colorbar=False)
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
#data.load()#["latent surface flux"].load()#plot()